### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

### Loading Dataset

In [2]:
def load_dataset(filename):
    # Reading the custom excel file designed for DURHAM COLLEGE COVID AWARENESS
    df = pd.read_excel(filename, names = ['Questions', 'Intents', 'Answer'])
    print(df.head())
    # Extract the Intents, Answers, and Questions
    intent = df['Intents']
    answer = df['Answer']
    # Extract unique Intents and Answers and zip them in Key-Value Pairs
    unique_intent = list(set(intent))
    unique_answers = list(set(answer))
    intent_answer_dict = dict(zip(unique_intent, unique_answers))
    questions = list(df['Questions'])

    return (intent, unique_intent, questions, intent_answer_dict)

In [3]:
intent, unique_intent, sentences, intent_answer_dict = load_dataset('covid-intents.xlsx')

                                           Questions    Intents  \
0                    Hello.. I need some information  greetings   
1           Hi.. I would like to get some assistance  greetings   
2  I want to know more about Durham College COVID...  greetings   
3  Hello Durham College COVID AWARENESS PROGRAM. ...  greetings   
4  Hello. I need help on Durham College COVID AWA...  greetings   

                                              Answer  
0  Hello!! Welcome to Durham College COVID AWAREN...  
1  Hello!! Welcome to Durham College COVID AWAREN...  
2  Hello!! Welcome to Durham College COVID AWAREN...  
3  Hello!! Welcome to Durham College COVID AWAREN...  
4  Hello!! Welcome to Durham College COVID AWAREN...  


In [4]:
print(sentences[:5])

['Hello.. I need some information', 'Hi.. I would like to get some assistance', 'I want to know more about Durham College COVID AWARENESS PROGRAM', 'Hello Durham College COVID AWARENESS PROGRAM. I would like to get some details', 'Hello. I need help on Durham College COVID AWARENESS PROGRAM']


### Data Preprocessing

In [5]:
# Define the stemmer
stemmer = LancasterStemmer()

In [6]:
def cleaning(sentences):
    words = []
    # Extract the sentences 
    for s in sentences:
        # Remove the punctuations
        clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
        # Extract the words from the sentence
        w = word_tokenize(clean)
        # Converting the words to Lower case and converting them back to a list
        words.append([i.lower() for i in w])

    return words

In [7]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])

86
[['hello', 'i', 'need', 'some', 'information'], ['hi', 'i', 'would', 'like', 'to', 'get', 'some', 'assistance']]


In [8]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
    # Create a tokenizer with the filters defined above
    token = Tokenizer(filters = filters)
    # Fit the created tokenizer on the sentences
    token.fit_on_texts(words)
    return token

In [9]:
def max_length(words):
    # Get the Maximum length of all the sentences
    return(len(max(words, key = len)))

In [10]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 191 and Maximum length = 22


In [11]:
def encoding_doc(token, words):
    # Encoding the list of words to list of integers based on BoW
    return(token.texts_to_sequences(words))

In [12]:
# Encoding the sentences
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [13]:
def padding_doc(encoded_doc, max_length):
    # Padding the sequence of integers to max-len of sentences
    return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [14]:
# Padding the sequences
padded_doc = padding_doc(encoded_doc, max_length)

In [15]:
padded_doc[:5]

array([[ 48,   1,  18,  32,  13,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [102,   1,  49,  50,   4,  51,  32,  52,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  1,  33,   4,  37, 103, 104,  12,  11,   2,   8,  20,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 48,  12,  11,   2,   8,  20,   1,  49,  50,   4,  51,  32, 105,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 48,   1,  18, 106,  53,  12,  11,   2,   8,  20,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [16]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (86, 22)


In [17]:
# Creating an output tokenizer
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')

In [18]:
# Intents
output_tokenizer.word_index

{'greetings': 1,
 'cost': 2,
 'latest_measures': 3,
 'student_infected': 4,
 'student_data_collection': 5,
 'set_up': 6,
 'customize': 7,
 'data_security': 8,
 'use': 9,
 'professor_infected': 10,
 'contact_tracing': 11,
 'support': 12,
 'consent': 13,
 'processing_time': 14,
 'cancel': 15}

In [19]:
# Encoding the outputs(Intents)
encoded_output = encoding_doc(output_tokenizer, intent)

In [20]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [21]:
encoded_output.shape

(86, 1)

In [22]:
def one_hot(encode):
    # One-hot-Encoding the intents
    o = OneHotEncoder(sparse = False)
    return(o.fit_transform(encode))

In [23]:
output_one_hot = one_hot(encoded_output)

In [24]:
output_one_hot.shape

(86, 15)

### Train-Test split

In [25]:
# Train-Test split of Dataset
from sklearn.model_selection import train_test_split
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)

In [26]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (68, 22) and train_Y = (68, 15)
Shape of val_X = (18, 22) and val_Y = (18, 15)


### Model Building and Evaluation

In [27]:
def create_model(vocab_size, max_length):
    # Creating a Sequential model
    model = Sequential()
    # Embdding layer with no. of neuron=vocab size
    model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
    # Bidirectional LSTM for better performance and understanding context
    model.add(Bidirectional(LSTM(128)))
    # Fully connected Dense layer
    model.add(Dense(32, activation = "relu"))
    # Dropout Layer to prevent Overfitting
    model.add(Dropout(0.5))
    # Dense output Layer with no. of Neurons=no. of Intents
    model.add(Dense(15, activation = "softmax"))

    return model

In [28]:
model = create_model(vocab_size, max_length)
# Create and compile the model
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 22, 128)           24448     
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               263168    
_________________________________________________________________
dense (Dense)                (None, 32)                8224      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                495       
Total params: 296,335
Trainable params: 271,887
Non-trainable params: 24,448
_________________________________________________________________


In [29]:
filename = 'intent-classifier.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# Fit and Save the model
hist = model.fit(train_X, train_Y, epochs = 1000, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

Epoch 1/1000
3/3 [==============================] - 7s 968ms/step - loss: 2.7070 - accuracy: 0.0191 - val_loss: 2.7253 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 2.72534, saving model to intent-classifier.h5
Epoch 2/1000
3/3 [==============================] - 0s 38ms/step - loss: 2.6990 - accuracy: 0.0715 - val_loss: 2.7464 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss did not improve from 2.72534
Epoch 3/1000
3/3 [==============================] - 0s 42ms/step - loss: 2.6910 - accuracy: 0.1170 - val_loss: 2.7739 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss did not improve from 2.72534
Epoch 4/1000
3/3 [==============================] - 0s 40ms/step - loss: 2.6755 - accuracy: 0.0715 - val_loss: 2.7981 - val_accuracy: 0.0000e+00

Epoch 00004: val_loss did not improve from 2.72534
Epoch 5/1000
3/3 [==============================] - 0s 41ms/step - loss: 2.7080 - accuracy: 0.0641 - val_loss: 2.8046 - val_accuracy: 0.0000e+00

Epoch 00005: val_loss did n


Epoch 00042: val_loss did not improve from 2.72534
Epoch 43/1000
3/3 [==============================] - 0s 35ms/step - loss: 2.3869 - accuracy: 0.2270 - val_loss: 2.8845 - val_accuracy: 0.0556

Epoch 00043: val_loss did not improve from 2.72534
Epoch 44/1000
3/3 [==============================] - 0s 35ms/step - loss: 2.2863 - accuracy: 0.1932 - val_loss: 2.8236 - val_accuracy: 0.0556

Epoch 00044: val_loss did not improve from 2.72534
Epoch 45/1000
3/3 [==============================] - 0s 40ms/step - loss: 2.3470 - accuracy: 0.2201 - val_loss: 2.7346 - val_accuracy: 0.0000e+00

Epoch 00045: val_loss did not improve from 2.72534
Epoch 46/1000
3/3 [==============================] - 0s 50ms/step - loss: 2.4119 - accuracy: 0.1893 - val_loss: 2.7610 - val_accuracy: 0.0556

Epoch 00046: val_loss did not improve from 2.72534
Epoch 47/1000
3/3 [==============================] - 0s 51ms/step - loss: 2.3521 - accuracy: 0.2691 - val_loss: 2.9192 - val_accuracy: 0.0556

Epoch 00047: val_loss did

3/3 [==============================] - 0s 35ms/step - loss: 1.7233 - accuracy: 0.4159 - val_loss: 3.0090 - val_accuracy: 0.1111

Epoch 00085: val_loss did not improve from 2.72534
Epoch 86/1000
3/3 [==============================] - 0s 34ms/step - loss: 1.8235 - accuracy: 0.3782 - val_loss: 3.1399 - val_accuracy: 0.1667

Epoch 00086: val_loss did not improve from 2.72534
Epoch 87/1000
3/3 [==============================] - 0s 36ms/step - loss: 1.6992 - accuracy: 0.4619 - val_loss: 3.2313 - val_accuracy: 0.1667

Epoch 00087: val_loss did not improve from 2.72534
Epoch 88/1000
3/3 [==============================] - 0s 36ms/step - loss: 1.6991 - accuracy: 0.3899 - val_loss: 3.0318 - val_accuracy: 0.1667

Epoch 00088: val_loss did not improve from 2.72534
Epoch 89/1000
3/3 [==============================] - 0s 37ms/step - loss: 1.6045 - accuracy: 0.4775 - val_loss: 2.8481 - val_accuracy: 0.1667

Epoch 00089: val_loss did not improve from 2.72534
Epoch 90/1000
3/3 [=========================


Epoch 00126: val_loss did not improve from 2.43776
Epoch 127/1000
3/3 [==============================] - 0s 37ms/step - loss: 1.2701 - accuracy: 0.6551 - val_loss: 2.9838 - val_accuracy: 0.2222

Epoch 00127: val_loss did not improve from 2.43776
Epoch 128/1000
3/3 [==============================] - 0s 35ms/step - loss: 1.2690 - accuracy: 0.5524 - val_loss: 3.0987 - val_accuracy: 0.2222

Epoch 00128: val_loss did not improve from 2.43776
Epoch 129/1000
3/3 [==============================] - 0s 37ms/step - loss: 1.1355 - accuracy: 0.6179 - val_loss: 3.5760 - val_accuracy: 0.2222

Epoch 00129: val_loss did not improve from 2.43776
Epoch 130/1000
3/3 [==============================] - 0s 76ms/step - loss: 1.3779 - accuracy: 0.5260 - val_loss: 3.7180 - val_accuracy: 0.2222

Epoch 00130: val_loss did not improve from 2.43776
Epoch 131/1000
3/3 [==============================] - 0s 62ms/step - loss: 1.0523 - accuracy: 0.6443 - val_loss: 2.9052 - val_accuracy: 0.1667

Epoch 00131: val_loss di


Epoch 00168: val_loss did not improve from 2.42027
Epoch 169/1000
3/3 [==============================] - 0s 37ms/step - loss: 0.9419 - accuracy: 0.6742 - val_loss: 2.5994 - val_accuracy: 0.3333

Epoch 00169: val_loss did not improve from 2.42027
Epoch 170/1000
3/3 [==============================] - 0s 33ms/step - loss: 1.0155 - accuracy: 0.6585 - val_loss: 2.6763 - val_accuracy: 0.3333

Epoch 00170: val_loss did not improve from 2.42027
Epoch 171/1000
3/3 [==============================] - 0s 37ms/step - loss: 0.7995 - accuracy: 0.7778 - val_loss: 3.9019 - val_accuracy: 0.2778

Epoch 00171: val_loss did not improve from 2.42027
Epoch 172/1000
3/3 [==============================] - 0s 36ms/step - loss: 0.9300 - accuracy: 0.6551 - val_loss: 4.6905 - val_accuracy: 0.2222

Epoch 00172: val_loss did not improve from 2.42027
Epoch 173/1000
3/3 [==============================] - 0s 43ms/step - loss: 1.1646 - accuracy: 0.6170 - val_loss: 2.7692 - val_accuracy: 0.2778

Epoch 00173: val_loss di

...

3/3 [==============================] - 0s 63ms/step - loss: 0.0285 - accuracy: 1.0000 - val_loss: 6.0918 - val_accuracy: 0.2778

Epoch 00630: val_loss did not improve from 2.41108
Epoch 631/1000
3/3 [==============================] - 0s 61ms/step - loss: 0.1102 - accuracy: 0.9584 - val_loss: 6.0509 - val_accuracy: 0.2778

Epoch 00631: val_loss did not improve from 2.41108
Epoch 632/1000
3/3 [==============================] - 0s 34ms/step - loss: 0.1121 - accuracy: 0.9432 - val_loss: 6.0243 - val_accuracy: 0.3333

Epoch 00632: val_loss did not improve from 2.41108
Epoch 633/1000
3/3 [==============================] - 0s 38ms/step - loss: 0.1149 - accuracy: 0.9550 - val_loss: 6.2459 - val_accuracy: 0.2778

Epoch 00633: val_loss did not improve from 2.41108
Epoch 634/1000
3/3 [==============================] - 0s 35ms/step - loss: 0.2449 - accuracy: 0.8977 - val_loss: 7.3234 - val_accuracy: 0.2778

Epoch 00634: val_loss did not improve from 2.41108
Epoch 635/1000
3/3 [====================

3/3 [==============================] - 0s 38ms/step - loss: 0.1085 - accuracy: 0.9432 - val_loss: 4.5766 - val_accuracy: 0.3889

Epoch 00714: val_loss did not improve from 2.41108
Epoch 715/1000
3/3 [==============================] - 0s 44ms/step - loss: 0.1852 - accuracy: 0.9315 - val_loss: 5.1418 - val_accuracy: 0.3889

Epoch 00715: val_loss did not improve from 2.41108
Epoch 716/1000
3/3 [==============================] - 0s 45ms/step - loss: 0.0749 - accuracy: 0.9584 - val_loss: 6.4412 - val_accuracy: 0.2778

Epoch 00716: val_loss did not improve from 2.41108
Epoch 717/1000
3/3 [==============================] - 0s 51ms/step - loss: 0.0872 - accuracy: 0.9623 - val_loss: 7.8111 - val_accuracy: 0.2778

Epoch 00717: val_loss did not improve from 2.41108
Epoch 718/1000
3/3 [==============================] - 0s 63ms/step - loss: 0.0845 - accuracy: 0.9887 - val_loss: 9.0821 - val_accuracy: 0.2778

Epoch 00718: val_loss did not improve from 2.41108
Epoch 719/1000
3/3 [====================

3/3 [==============================] - 0s 71ms/step - loss: 0.3927 - accuracy: 0.8709 - val_loss: 7.0566 - val_accuracy: 0.1667

Epoch 00756: val_loss did not improve from 2.41108
Epoch 757/1000
3/3 [==============================] - 0s 71ms/step - loss: 0.4788 - accuracy: 0.8674 - val_loss: 6.4410 - val_accuracy: 0.2222

Epoch 00757: val_loss did not improve from 2.41108
Epoch 758/1000
3/3 [==============================] - 0s 71ms/step - loss: 0.3226 - accuracy: 0.9285 - val_loss: 5.5730 - val_accuracy: 0.2222

Epoch 00758: val_loss did not improve from 2.41108
Epoch 759/1000
3/3 [==============================] - 0s 70ms/step - loss: 0.2131 - accuracy: 0.9697 - val_loss: 5.0244 - val_accuracy: 0.2222

Epoch 00759: val_loss did not improve from 2.41108
Epoch 760/1000
3/3 [==============================] - 0s 70ms/step - loss: 0.2098 - accuracy: 0.9281 - val_loss: 4.4963 - val_accuracy: 0.2222

Epoch 00760: val_loss did not improve from 2.41108
Epoch 761/1000
3/3 [====================

3/3 [==============================] - 0s 40ms/step - loss: 0.1586 - accuracy: 0.9281 - val_loss: 5.7255 - val_accuracy: 0.4444

Epoch 00840: val_loss did not improve from 2.41108
Epoch 841/1000
3/3 [==============================] - 0s 35ms/step - loss: 0.0782 - accuracy: 0.9775 - val_loss: 5.6549 - val_accuracy: 0.3889

Epoch 00841: val_loss did not improve from 2.41108
Epoch 842/1000
3/3 [==============================] - 0s 40ms/step - loss: 0.1073 - accuracy: 0.9736 - val_loss: 5.6792 - val_accuracy: 0.3889

Epoch 00842: val_loss did not improve from 2.41108
Epoch 843/1000
3/3 [==============================] - 0s 34ms/step - loss: 0.1242 - accuracy: 0.9506 - val_loss: 5.7465 - val_accuracy: 0.3889

Epoch 00843: val_loss did not improve from 2.41108
Epoch 844/1000
3/3 [==============================] - 0s 34ms/step - loss: 0.2287 - accuracy: 0.9203 - val_loss: 5.6840 - val_accuracy: 0.3333

Epoch 00844: val_loss did not improve from 2.41108
Epoch 845/1000
3/3 [====================

3/3 [==============================] - 0s 64ms/step - loss: 0.1205 - accuracy: 0.9506 - val_loss: 6.6568 - val_accuracy: 0.2778

Epoch 00924: val_loss did not improve from 2.41108
Epoch 925/1000
3/3 [==============================] - 0s 66ms/step - loss: 0.0971 - accuracy: 0.9809 - val_loss: 6.7473 - val_accuracy: 0.2778

Epoch 00925: val_loss did not improve from 2.41108
Epoch 926/1000
3/3 [==============================] - 0s 69ms/step - loss: 0.0974 - accuracy: 0.9697 - val_loss: 6.8420 - val_accuracy: 0.2778

Epoch 00926: val_loss did not improve from 2.41108
Epoch 927/1000
3/3 [==============================] - 0s 70ms/step - loss: 0.0944 - accuracy: 0.9697 - val_loss: 7.0167 - val_accuracy: 0.2778

Epoch 00927: val_loss did not improve from 2.41108
Epoch 928/1000
3/3 [==============================] - 0s 70ms/step - loss: 0.1106 - accuracy: 0.9623 - val_loss: 6.9845 - val_accuracy: 0.2778

Epoch 00928: val_loss did not improve from 2.41108
Epoch 929/1000
3/3 [====================

In [30]:
# Load the model
model = load_model('intent-classifier.h5')

### Model Testing

In [31]:
# Function to Preprocess the manual user inputs
def predictions(text):
    # Remove Punctuations
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
    # Extract the words from the sentence
    test_word = word_tokenize(clean)
    # Converting the words to Lower case and converting them back to a list
    test_word = [w.lower() for w in test_word]
    # Encode sentences to sequences
    test_ls = word_tokenizer.texts_to_sequences(test_word)
    print(test_word)
    # Check for unknown words not in the actual BoW
    if [] in test_ls:
        test_ls = list(filter(None, test_ls))

    test_ls = np.array(test_ls).reshape(1, len(test_ls))
    # Pad the sequences to max length
    x = padding_doc(test_ls, max_length)
    # Predict the Intent
    pred = model.predict_proba(x)


    return pred

In [32]:
def get_final_output(pred, classes):
    # Get the Prediction scores 
    predictions = pred[0]
    # Get all the unique Intents
    classes = np.array(classes)
    # Sort the prediction scores for all Intents
    ids = np.argsort(-predictions)
    # Sort all the Intents according to the scores using sorted indexes
    classes = classes[ids]
    predictions = -np.sort(-predictions)
    # Select the Top Intent
    predicted_intent = classes[0]
    #for i in range(pred.shape[1]):
    #    print("%s has confidence = %s" % (classes[i], (predictions[i])))
        
    # Select the Answer for the Intent selected (Key-value pair) 
    print(intent_answer_dict[predicted_intent])

In [33]:
text = "I would like to get some technical assistance."
pred = predictions(text)
#print(pred)
get_final_output(pred, unique_intent)

['i', 'would', 'like', 'to', 'get', 'some', 'technical', 'assistance']


C:\Users\rosha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Depending on the province, it is either mandatory or recommended for public health. We believe that by collecting customer data we can help to keep Canadians safe and prevent further spread. For more information you can visit your provincial or municipal website.
